<a href="https://colab.research.google.com/github/IhkafAmeny/ABIDE-Dataset/blob/main/nii_to_png_converter_ABIDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 Installer les bibliothèques nécessaires
!pip install nibabel pillow tqdm --quiet

# 📚 Importer les bibliothèques
import os
import nibabel as nib
import numpy as np
from PIL import Image
from tqdm import tqdm
from google.colab import drive, files

drive.mount('/content/drive')

input_folder = '/content/drive/MyDrive/Master Ameny Ihkaf/abide dataset/abide1000/image'
output_folder = '/content/abide_png_output'
os.makedirs(output_folder, exist_ok=True)


def normalize_image(data):
    data = data.astype(np.float32)
    min_val = np.min(data)
    max_val = np.max(data)

    if max_val != min_val:
        data = (data - min_val) / (max_val - min_val) * 255
    else:
        data = np.zeros_like(data)

    return data.astype(np.uint8)

nii_files = [f for f in os.listdir(input_folder) if f.endswith('.nii') or f.endswith('.nii.gz')]
print(f"Found {len(nii_files)} NIfTI files.")

processed_count = 0
error_count = 0

for filename in tqdm(nii_files, desc="Converting NIfTI to PNG"):
    filepath = os.path.join(input_folder, filename)

    try:
        img = nib.load(filepath)
        data = img.get_fdata()

        # Nom de fichier PNG
        if filename.endswith('.nii.gz'):
            png_name = filename[:-7] + '.png'
        elif filename.endswith('.nii'):
            png_name = filename[:-4] + '.png'
        else:
            png_name = filename + '.png'

        # Gestion des dimensions
        if len(data.shape) == 4:
            data_3d = np.mean(data, axis=3)
            center_slice = data_3d[:, :, data_3d.shape[2] // 2]
        elif len(data.shape) == 3:
            center_slice = data[:, :, data.shape[2] // 2]
        elif len(data.shape) == 2:
            center_slice = data
        else:
            print(f"IGNORED: {filename} (unsupported shape {data.shape})")
            continue

        # Normaliser et sauvegarder
        slice_normalized = normalize_image(center_slice)
        slice_normalized = np.rot90(slice_normalized)  # orienter correctement
        img_pil = Image.fromarray(slice_normalized)

        output_path = os.path.join(output_folder, png_name)
        img_pil.save(output_path)

        processed_count += 1

    except Exception as e:
        print(f"ERROR with {filename}: {e}")
        error_count += 1

example_files = os.listdir(output_folder)
if example_files:
    print(f"\nSample PNG: {example_files[0]}")
    files.download(os.path.join(output_folder, example_files[0]))
